In [3]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor, ViTModel, ViTConfig, CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer, models
from transformers import AutoImageProcessor, AutoModel
from transformers import ConvNextImageProcessor, ConvNextForImageClassification
from nltk.corpus import wordnet as wn
import pandas as pd

import torch, random
from tqdm import tqdm
import sys
import re
import os
import json

In [44]:
all_assigments = []
all_df = {}
sample_to_part = {}

for part in tqdm(range(12)):
        
    f = open(f'results_final/{part}_assignment_0.6_1.json')
    class_assigments = json.load(f)[:-1]

    filename = f'/shared/raiymbek/vlm_2/laion400m-meta/part-{part:05d}-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet'
    df = pd.read_parquet(filename)
    df = df.dropna(subset=['TEXT'])
    df_size = df.shape[0]

    for cl in class_assigments:
        for i in cl:
            sample_to_part[i[0]] = part
            part_to_sample[part].append(i[0])

    all_df[part] = df
    all_assigments.append(class_assigments)

final_assignments = [[] for i in range(1000)]
for assignment in all_assigments:
    for i in range(1000):
        final_assignments[i] += assignment[i]

picked_assignments = [sorted(i, key = lambda x: -x[1])[:2000] for i in final_assignments]
picked_assignments = [[j for j in i if not pd.isna(j[0])][:1000] for i in picked_assignments]

part_to_sample = [[] for i in range(12)]
sample_to_class = {}
sample_to_sim = {}
for cl_i in range(1000):
    cl = picked_assignments[cl_i]
    for i in cl:
        part = sample_to_part[i[0]]
        part_to_sample[part].append(i[0])
        sample_to_class[i[0]] = cl_i
        sample_to_sim[i[0]] = i[1]

table = []
for part in tqdm(range(12)):
    sample_df = all_df[part]
    part_samples = part_to_sample[part]
    picked_df = sample_df[sample_df["SAMPLE_ID"].isin(part_samples)].values.tolist()
    for i in range(len(picked_df)):
        row = picked_df[i].copy()
        row.append(sample_to_class[row[0]])
        row.append(sample_to_sim[row[0]])
        table.append(row)

100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:25<00:00,  2.11s/it]


In [49]:
df = pd.DataFrame(table, columns = list(de.head().columns) + ["IMAGENET_CLASS", "CLASS_SIM"])

In [50]:
df.to_parquet('collection.snappy.parquet')

In [51]:
pd.read_parquet('collection.snappy.parquet')

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity,IMAGENET_CLASS,CLASS_SIM
0,3.820200e+11,https://t2.ftcdn.net/jpg/00/58/35/35/240_F_583...,Acute pain in a woman knee,257.0,240.0,?,UNLIKELY,0.344278,615,0.657821
1,1.476880e+11,http://d25hqtnqp5nl24.cloudfront.net/images/pr...,Actimel vanilla - 8x100g Brand Price Match - C...,140.0,140.0,?,UNLIKELY,0.401302,358,0.181231
2,3.640454e+12,https://www.nutramini.com/wp-content/uploads/2...,Essence-of-Argan,350.0,220.0,?,UNSURE,0.340446,711,0.776829
3,1.061189e+12,http://thumbs.slideserve.com/1_6090843.jpg,IAM Overview and Self-assessment Exercise,320.0,240.0,?,UNLIKELY,0.385074,824,0.728641
4,2.929264e+12,https://16zypw1pighqx2n3o4ber801-wpengine.netd...,Turquoise Cross - Turquoise Inlay Cross Pendan...,416.0,312.0,?,UNLIKELY,0.369402,451,0.223459
...,...,...,...,...,...,...,...,...,...,...
988838,1.595157e+12,https://images1.americanlisted.com/nlarge/cute...,Cute Shih-Tzu Puppy,640.0,480.0,?,UNLIKELY,0.300886,204,0.740261
988839,3.326491e+12,https://dy6g3i6a1660s.cloudfront.net/dEWOXq8YR...,Caiman Lizard Inspired,300.0,277.0,?,UNLIKELY,0.321765,44,0.712954
988840,3.943553e+12,https://cdn.shopify.com/s/files/1/0275/0309/99...,Ecosystem Information Boards - Orb Spider,300.0,200.0,?,UNLIKELY,0.300037,72,0.601161
988841,7.191601e+10,http://d1hw6n3yxknhky.cloudfront.net/008960995...,Stock Video Footage of proboscis monkey eating 2,188.0,104.0,?,UNLIKELY,0.328545,376,0.703380


In [52]:
de.head()

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity
0,1.581282e+12,http://media.rightmove.co.uk/148k/147518/58718...,View EPC Rating Graph for this property,109.0,100.0,?,UNSURE,0.312813
1,1.060015e+12,https://thumbs.ebaystatic.com/images/g/DYEAAOS...,Silverline Air Framing Nailer 90mm 10 - 12 Gau...,225.0,225.0,?,UNLIKELY,0.312485
2,3.372497e+12,https://farm1.staticflickr.com/784/40182677504...,Anhui Mountains,800.0,514.0,?,UNLIKELY,0.316512
3,3.820200e+11,https://t2.ftcdn.net/jpg/00/58/35/35/240_F_583...,Acute pain in a woman knee,257.0,240.0,?,UNLIKELY,0.344278
4,2.928456e+12,https://findingblanche.files.wordpress.com/201...,Venison – Sour Cherries – Cream – Potato,764.0,577.0,?,NSFW,0.304397
...,...,...,...,...,...,...,...,...
12933519,1.539700e+10,http://s5.favim.com/orig/53/beautiful-black-an...,"beautiful, black and white, fashion, heel",500.0,333.0,?,UNLIKELY,0.338823
12933520,3.990059e+12,https://www.hobbidee-breda.nl/wp-content/uploa...,Jilliez Artliners - Colour,247.0,300.0,?,UNLIKELY,0.358599
12933521,3.405165e+12,https://cdn.shopify.com/s/files/1/0142/2173/80...,#1 Mom Heart Pendant,300.0,300.0,?,UNLIKELY,0.311529
12933522,1.394591e+12,https://3.bp.blogspot.com/-6M42JtD_Ht8/Ut__TVV...,Wedding entertainment North East corporate eve...,1131.0,1600.0,?,UNSURE,0.351486
